# GPT로 이미지 설명 만들기

In [1]:
import base64
from openai import OpenAI
from src.utils import get_openai_api_key

In [2]:
# OpenAI 객체 생성
client = OpenAI(api_key=get_openai_api_key())

## [Lorem Picsum](https://picsum.photos) 이미지 설명 요청

In [3]:
# 이미지 설명을 요청하는 메시지 프롬프트 작성
messages = [
    {
        'role': 'user',
        'content': [
            {'type': 'text', 'text': '이 이미지를 설명해줘.'},
            {'type': 'image_url',
             'image_url': {'url': 'https://fastly.picsum.photos/id/22/4434/3729.jpg?hmac=fjZdkSMZJNFgsoDh8Qo5zdA_nSGUAWvKLyyqmEt2xs0'}},
        ],
    },
]

In [4]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

In [5]:
response

ChatCompletion(id='chatcmpl-CDPMBacDpQMUnCQ4UsxJKsCv2LWPF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이미지에는 정장을 입은 한 남성이 도로 위를 걸어가고 있는 모습이 담겨 있습니다. 도로는 크랙이 있는 아스팔트로 보이며, 흰색 차선이 보입니다. 이 남성은 도시 환경에서 혼자 걷고 있는 듯하며, 전체적으로 차분하고 정적인 분위기를 자아냅니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757311771, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_e665f7564b', usage=CompletionUsage(completion_tokens=79, prompt_tokens=25515, total_tokens=25594, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [6]:
response.choices[0].message.content

'이미지에는 정장을 입은 한 남성이 도로 위를 걸어가고 있는 모습이 담겨 있습니다. 도로는 크랙이 있는 아스팔트로 보이며, 흰색 차선이 보입니다. 이 남성은 도시 환경에서 혼자 걷고 있는 듯하며, 전체적으로 차분하고 정적인 분위기를 자아냅니다.'

## 로컬 이미지 파일의 설명 요청하기

*   이미지 바이너리 데이터를 텍스트 형식으로 변환해서 GPT chat.completions 요청을 보내야 함.
*   base64 라이브러리: 이진(binary) 데이터를 아스키(ASCII) 문자로 인코딩(변환).
    *   전송/저장 호환성
    *   데이터 손상 방지
    *   프로토콜 호환성

In [7]:
def base64encode_image(image_file):
    with open(file=image_file, mode='rb') as f:  # 이미지 파일을 오픈.
        data = f.read()  # 이미지 파일의 이진 데이터를 읽음.

        # 이진 데이터를 base64 인코딩을 해서 ASCII 문자열로 변환, UTF-8로 변환해서 리턴.
        return base64.b64encode(data).decode(encoding='utf-8')

In [8]:
image_path_1 = './images/cafe1.jpg'
image_path_2 = './images/cafe2.jpg'

In [9]:
base64_image_1 = base64encode_image(image_path_1)

In [11]:
base64_image_1[:100]  # base64로 인코딩된 문자열들 중에서 앞에서 100개만 출력.

'/9j/4AAQSkZJRgABAgEASABIAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABh'

In [12]:
# GPT에게 보낼 메시지 프롬프트 작성
messages = [
    {
        'role': 'user',
        'content': [
            {'type': 'text', 'text': '이미지에 대해서 설명해줘.'},
            {
                'type': 'image_url',
                'image_url': {'url': f'data:image/jpeg;base64,{base64_image_1}'}
            },
        ],
    },
]

In [13]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

In [14]:
response.choices[0].message.content

'이 이미지는 밝고 현대적인 레스토랑이나 카페의 내부를 보여줍니다. 큰 유리 천장은 자연광을 충분히 받아들이고 있으며, 여러 종류의 식물이 공간을 아늑하게 꾸미고 있습니다. 식탁과 의자가 깔끔하게 정돈되어 있고, 몇몇 손님들이 대화를 나누거나 음료를 즐기고 있는 모습이 보입니다. 바닥은 콘크리트로 되어 있으며, 벽면에는 바리스타가 사용하는 기계와 음료가 놓여 있는 바 공간이 있습니다. 아늑하고 편안한 분위기로, 식물과 자연 요소들이 강조된 인테리어가 인상적입니다.'

이미지 2장을 비교/설명 요청하는 메시지 프롬프트를 작성하고, 실행 결과를 확인해 보세요.